In [1]:
import pandas as pd

In [53]:
# 이렇게 판다스는 json파일도 불러올 수 있습니다.
nobel = pd.read_json('nobel.json')
nobel.head()

,prizes
0,"{'year': '2019', 'category': 'chemistry', 'lau..."
1,"{'year': '2019', 'category': 'economics', 'lau..."
2,"{'year': '2019', 'category': 'literature', 'la..."
3,"{'year': '2019', 'category': 'peace', 'laureat..."
4,"{'year': '2019', 'category': 'physics', 'overa..."


In [54]:
# 이렇게 Pizes열만 한번 가져와보죠.
chemistry_2019 = nobel.loc[0, 'prizes']
chemistry_2019

{'year': '2019',
 'category': 'chemistry',
 'laureates': [{'id': '976',
   'firstname': 'John',
   'surname': 'Goodenough',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'},
  {'id': '977',
   'firstname': 'M. Stanley',
   'surname': 'Whittingham',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'},
  {'id': '978',
   'firstname': 'Akira',
   'surname': 'Yoshino',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'}]}

In [56]:
# json 형태의 파일을 이렇게 평탄화 시킬 수 있습니다.
pd.json_normalize(data=chemistry_2019)
# 하지만, laureates 부분에서 아직 딕셔너리 형태가 되어있네요.

,year,category,laureates
0,2019,chemistry,"[{'id': '976', 'firstname': 'John', 'surname':..."


In [57]:
# record_path로 정규화시켜주고 싶은 부분을 입력하면, 다시 세부적으로 나뉘는 것을 알 수 있습니다.
pd.json_normalize(data=chemistry_2019,
                  record_path='laureates')

,id,firstname,surname,motivation,share
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3


In [58]:
# 하지만, 기존에 year과 firstName의 데이터가 사라졌네요?
# 이럴때는 meta에 year과 firstname을 지정해 해당 데이터프레임을 구성할 수 있습니다.
pd.json_normalize(data=chemistry_2019,
                  meta=['year', 'category'],
                  record_path='laureates')

,id,firstname,surname,motivation,share,year,category
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3,2019,chemistry
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3,2019,chemistry
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3,2019,chemistry


In [59]:
# 그렇다면 이번에는 noble상의 전체데이터에 적용해보죠.
pd.json_normalize(data=nobel['prizes'],
                  meta=['year', 'category'],
                  record_path='laureates')

KeyError: "Key 'laureates' not found. If specifying a record_path, all elements of data should have the path."

In [60]:
# 오류가 발생합니다! laurates가 없는 행이 있다는 이야긴데요?
# 어떻게 해결해야할까요?

# 아래 데이터를 한번 볼게요.
cheese_consumption = {
    'France':57.9,
    'Germany':53.2,
    'Luxembourg':53.2
}

In [61]:
# setdefault를 통해서 키:값의 데이터를 넣어줄 수 있습니다.
# 하지만, 프랑스에 대한 키값이 이미 있기때문에 기존 값을 그대로 반환하죠.
cheese_consumption.setdefault('France', 100)

57.9

In [62]:
# 만약, 키값이 없는 값은 어떻게 될까요?
cheese_consumption.setdefault('Italy', 48)
cheese_consumption

{'France': 57.9, 'Germany': 53.2, 'Luxembourg': 53.2, 'Italy': 48}

In [63]:
# 보시다시피 키값이 없는 값은 대입하고, 그렇지 않은 데이터는 기존값을 반환하군요.

In [64]:
# 그렇다면 우리가 함수를 만들어 봅시다.
# 기존에 laureates를 가지고 있다면 기존 값을 반환할테지만, 없다면 빈리스트를 가지게되겠죠?
def add_laurates_key(entry):
    entry.setdefault('laureates', [])
nobel['prizes'].apply(add_laurates_key)

0      None
1      None
2      None
3      None
4      None
       ... 
641    None
642    None
643    None
644    None
645    None
Name: prizes, Length: 646, dtype: object

In [65]:
# 이제, 다시 정규화를 시켜봆시다.
winners = pd.json_normalize(data=nobel['prizes'],
                            meta=['year', 'category'],
                            record_path='laureates')
winners

,id,firstname,surname,motivation,share,year,category
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3,2019,chemistry
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3,2019,chemistry
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3,2019,chemistry
3,982,Abhijit,Banerjee,"""for their experimental approach to alleviatin...",3,2019,economics
4,983,Esther,Duflo,"""for their experimental approach to alleviatin...",3,2019,economics
...,...,...,...,...,...,...,...
945,569,Sully,Prudhomme,"""in special recognition of his poetic composit...",1,1901,literature
946,462,Henry,Dunant,"""for his humanitarian efforts to help wounded ...",2,1901,peace
947,463,Frédéric,Passy,"""for his lifelong work for international peace...",2,1901,peace
948,1,Wilhelm Conrad,Röntgen,"""in recognition of the extraordinary services ...",1,1901,physics


In [66]:
# records를 사용하면, 키-값으로 변환합니다.
winners.to_json(orient='records')

'[{"id":"976","firstname":"John","surname":"Goodenough","motivation":"\\"for the development of lithium-ion batteries\\"","share":"3","year":"2019","category":"chemistry"},{"id":"977","firstname":"M. Stanley","surname":"Whittingham","motivation":"\\"for the development of lithium-ion batteries\\"","share":"3","year":"2019","category":"chemistry"},{"id":"978","firstname":"Akira","surname":"Yoshino","motivation":"\\"for the development of lithium-ion batteries\\"","share":"3","year":"2019","category":"chemistry"},{"id":"982","firstname":"Abhijit","surname":"Banerjee","motivation":"\\"for their experimental approach to alleviating global poverty\\"","share":"3","year":"2019","category":"economics"},{"id":"983","firstname":"Esther","surname":"Duflo","motivation":"\\"for their experimental approach to alleviating global poverty\\"","share":"3","year":"2019","category":"economics"},{"id":"984","firstname":"Michael","surname":"Kremer","motivation":"\\"for their experimental approach to allevi

In [67]:
# 반대로 split은 별도의 열, 인덱스, 키가 있는 딕셔너리를 반환합니다.
winners.to_json(orient='split')

'{"columns":["id","firstname","surname","motivation","share","year","category"],"index":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,2

In [68]:
# 아기 이름데이터를 불러옵시다.
url = "https://data.cityofnewyork.us/api/views/25th-nujf/rows.csv"
baby_names = pd.read_csv(url)
baby_names.head()

,Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank
0,2011,FEMALE,HISPANIC,GERALDINE,13,75
1,2011,FEMALE,HISPANIC,GIA,21,67
2,2011,FEMALE,HISPANIC,GIANNA,49,42
3,2011,FEMALE,HISPANIC,GISELLE,38,51
4,2011,FEMALE,HISPANIC,GRACE,36,53


In [69]:
# csv는 기본적으로 쉼표(,)를 기준으로 데이터가 나뉩니다. 그리고 \n을 기준으로 다음행으로 변경되죠.
baby_names.head().to_csv()

",Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank\n0,2011,FEMALE,HISPANIC,GERALDINE,13,75\n1,2011,FEMALE,HISPANIC,GIA,21,67\n2,2011,FEMALE,HISPANIC,GIANNA,49,42\n3,2011,FEMALE,HISPANIC,GISELLE,38,51\n4,2011,FEMALE,HISPANIC,GRACE,36,53\n"

In [70]:
# index의 값을 False로 지정한다면, 0, 1, 2, 와 같은 값을 사용하지 않고 \n만을 반환합니다.
baby_names.head().to_csv(index=False)

"Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank\n2011,FEMALE,HISPANIC,GERALDINE,13,75\n2011,FEMALE,HISPANIC,GIA,21,67\n2011,FEMALE,HISPANIC,GIANNA,49,42\n2011,FEMALE,HISPANIC,GISELLE,38,51\n2011,FEMALE,HISPANIC,GRACE,36,53\n"

In [71]:
# 이제 해당 데이터를 저장해보죠.
baby_names.to_csv('NYC_Baby_Names.csv', index=False)

In [72]:
# 판다스에서는 엑셀 데이터를 다루기 위해서 두가지 라이브러리가 필요합니다.
# openpyxl과 xlrd를 꼭 설치해주세요!
pd.read_excel('Single Worksheet.xlsx')

,First Name,Last Name,City,Gender
0,Brandon,James,Miami,M
1,Sean,Hawkins,Denver,M
2,Judy,Day,Los Angeles,F
3,Ashley,Ruiz,San Francisco,F
4,Stephanie,Gomez,Portland,F


In [73]:
# csv 사용방법과 유사합니다.
pd.read_excel(io='Single Worksheet.xlsx',
              usecols=['First Name', 'Last Name', 'City'],
              index_col='City')

,First Name,Last Name
City,,
Miami,Brandon,James
Denver,Sean,Hawkins
Los Angeles,Judy,Day
San Francisco,Ashley,Ruiz
Portland,Stephanie,Gomez


In [74]:
# 이렇게 다른 시트를 가지는 데이터도 불러 올 수 있는데
pd.read_excel(io='Multiple Worksheets.xlsx')

,First Name,Last Name,City,Gender
0,Brandon,James,Miami,M
1,Sean,Hawkins,Denver,M
2,Judy,Day,Los Angeles,F
3,Ashley,Ruiz,San Francisco,F
4,Stephanie,Gomez,Portland,F


In [75]:
# sheet_name을 인덱스로 설정하던지, 이름으로 직접 설정하여 불러올 수 있습니다.
pd.read_excel(io='Multiple Worksheets.xlsx',
              sheet_name=1)

,First Name,Last Name,City,Gender
0,Parker,Power,Raleigh,F
1,Preston,Prescott,Philadelphia,F
2,Ronaldo,Donaldo,Bangor,M
3,Megan,Stiller,San Francisco,M
4,Bustin,Jieber,Austin,F


In [76]:
# sheet_name을 None으로 설정하면 모든 사트지를 불러오고
workbook = pd.read_excel('Multiple Worksheets.xlsx', sheet_name=None)
workbook

{'Data 1':   First Name Last Name           City Gender
 0    Brandon     James          Miami      M
 1       Sean   Hawkins         Denver      M
 2       Judy       Day    Los Angeles      F
 3     Ashley      Ruiz  San Francisco      F
 4  Stephanie     Gomez       Portland      F,
 'Data 2':   First Name Last Name           City Gender
 0     Parker     Power        Raleigh      F
 1    Preston  Prescott   Philadelphia      F
 2    Ronaldo   Donaldo         Bangor      M
 3      Megan   Stiller  San Francisco      M
 4     Bustin    Jieber         Austin      F,
 'Data 3':   First Name  Last Name     City Gender
 0     Robert     Miller  Seattle      M
 1       Tara     Garcia  Phoenix      F
 2    Raphael  Rodriguez  Orlando      M}

In [77]:
# 특정 시트만 불러올 수 있습니다.
workbook['Data 3']

,First Name,Last Name,City,Gender
0,Robert,Miller,Seattle,M
1,Tara,Garcia,Phoenix,F
2,Raphael,Rodriguez,Orlando,M


In [78]:
# 다시 아기이름 데이터를 한번 볼까요?
baby_names.head()

,Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank
0,2011,FEMALE,HISPANIC,GERALDINE,13,75
1,2011,FEMALE,HISPANIC,GIA,21,67
2,2011,FEMALE,HISPANIC,GIANNA,49,42
3,2011,FEMALE,HISPANIC,GISELLE,38,51
4,2011,FEMALE,HISPANIC,GRACE,36,53


In [79]:
# 여자아이와, 남자아이의 데이터를 분류하고 엑셀 파일에 시트를 다르게 저장해봅시다.
girls = baby_names[baby_names['Gender']=='FEMALE']
boys = baby_names[baby_names['Gender']=='MALE']

In [80]:
# csv의 파일의 경우 to_csv로 그냥 저장이 되었지만, excel 파일의 경우 ExcelWriter 과정을 한번 더 거쳐야합니다.
excel_file = pd.ExcelWriter('Baby_Names.xlsx')
excel_file

In [81]:
girls.to_excel(excel_writer=excel_file,
               sheet_name='Girl',
               index=False)

In [82]:
boys.to_excel(excel_writer=excel_file,
              sheet_name='Boys',
              index=False,
              columns=["Child's First Name", 'Count', 'Rank'])

In [84]:
# close()함수를 사용해주면 지금까지 저장된 시트를 엑셀 파일로 저장합니다.
excel_file.close()